In [11]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re

In [38]:
data=pd.read_csv("scisumm.csv",nrows = 201)

In [39]:
stopWords = set(stopwords.words("english"))
ownwords = ["to","the","for","fi","ff","zk","x","fifi","f","fl","of","for","is","are","for","to","in","and","at","with","acc","it"]
Prepositions = ['about','above','across','after','against','along','among','around','at','before','behind','between','beyond','but','by','concerning','despite','down','during','except','following','for','from','in','including','into','like','near','of','off','on','onto','out','over','past','plus','since','throughout','to','towards','under','until','up','upon','up','to','with','within','without']
stopWords.update(ownwords)
stopWords.update(Prepositions)

In [40]:
def TextCleaning(text,num):
    text = text.lower()
    cleantext = re.sub("\(.*?\)", '', text)
    cleantext = re.sub("[0-9]", '', cleantext)
    cleantext = re.sub("(\.\.+)", ' ',cleantext)
    cleantext = re.sub("(--+)", ' ',cleantext)
    cleantext = re.sub("(~~+)", ' ',cleantext)
    cleantext = re.sub("[<>()|&©ø\[\]\'\";~*]", ' ',cleantext)
    cleantext = re.sub("(\+\++)", ' ',cleantext)
    cleantext = re.sub("(__+)", ' ',cleantext)
    cleantext = re.sub("e.g.", '',cleantext)
    cleantext = re.sub("i.e.,", '',cleantext)
    cleantext = re.sub("acc.", '',cleantext)
    #cleantext = re.sub("[^a-zA-Z]", ' ',cleantext)
    cleantext = re.sub("(\s+)",' ',cleantext)
    if(num==0):
        tokens = [w for w in cleantext.split() if not w in stopWords]
    else:
        tokens=cleantext.split()
    long_words=[]
    for i in tokens:
        if len(i)>2:                                              
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [41]:
cleaned_text = []
for t in data['text']:
    cleaned_text.append(TextCleaning(t,2)) 
#call the function
cleaned_summary = []
for t in data['summary']:
    cleaned_summary.append(TextCleaning(t,0))
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary
data.dropna(axis=0,inplace=True)
print(type(data["cleaned_summary"][0]))

<class 'str'>


In [61]:
allsum = []
for text in data["cleaned_text"]:
    words = word_tokenize(text)
    words = word_tokenize(text)
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1
    sentences = sent_tokenize(text)
    sentenceValue = dict()
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq
    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]
 
    average = int(sumValues / len(sentenceValue))
    summary = ''
    sencont = 0
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.6 * average)):
            sencont+=1
            summary += " " + sentence
    allsum.append(summary)

#data["Result1"] = allsum


In [63]:
data["Result"] = allsum

In [65]:
from rouge_score import rouge_scorer

reference = data['cleaned_summary']
hypotheses = data["Result"]

hypotheses = str(hypotheses)
reference = str(reference)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(hypotheses,
                      reference)
print(hypotheses)
print("____________________")
print(reference)
print(scores)

0       additionally, present results the tagger the ...
1       section define and compare five different con...
2       describe statistical classifier that combines...
3       given input sentence and target word frame, t...
4       the potential benefit wide-coverage parsing w...
                             ...                        
196     they train from the penn treebank collection ...
197     co-training weakly supervised paradigm for le...
198     the unsupervised systems fact, all the seven ...
199     japanese dependency structure analysis based ...
200     syntactic reordering approaches are effective...
Name: Result, Length: 201, dtype: object
____________________
0      tnt statistical part-of-speech tagger trigrams...
1      mildly non-projective dependency structures sy...
2      using corpus statistics wordnet relations sens...
3      automatic labeling semantic roles present syst...
4      generative models statistical parsing combinat...
                          

In [140]:
df=pd.read_csv("scisumm.csv",nrows = 201)
cleaned_text = []
for t in df['text']:
    cleaned_text.append(TextCleaning(t,2)) 
#call the function
cleaned_summary = []
for t in df['summary']:
    cleaned_summary.append(TextCleaning(t,0))
df['cleaned_text']=cleaned_text
df['cleaned_summary']=cleaned_summary
data.dropna(axis=0,inplace=True)

In [141]:
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')

def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stopWords])
        return sen_new
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

allsum2 = []

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cll48\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
for text in df["cleaned_text"]:
    sentences = []
    sentences.append(sent_tokenize(text))
    sentences = [y for x in sentences for y in x]
    clean_sentences = pd.Series(sentences)
    clean_sentences = [s.lower() for s in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
        
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph,max_iter=10)

    #Summary Extraction
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    # Extract top 10 sentences as the summary
    summary2 = []
    for i in range(4):
        summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
    summary2 = str(summary2)
    allsum2.append(summary2)

In [90]:
reference = data['cleaned_summary'][0:50]
hypotheses = allsum2[0:50]

hypotheses = str(hypotheses)
hypotheses = re.sub("', '", '', hypotheses)
reference = str(reference)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(hypotheses,
                      reference)
print(hypotheses)
print("____________________")
print(reference)
print(scores)

[['It turned out to be a good choice to set all 0, to the standard deviation of the unconditioned maximum likelihood probabilities of the tags in the training corpus, i.e., we set for all i = 0 ... m — 1, using a tagset of s tags and the average (11) This usually yields values in the range 0,03 ... 0.10.The rather large amount of freedom was not handled in detail in previous publications: handling of start- and end-of-sequence, the exact smoothing technique, how to determine the weights for context probabilities, details on handling unknown words, and how to determine the weights for unknown words.It causes the probability of a complete sequence to be set to zero if its use is necessary for a new text sequence, thus makes it impossible to rank different sequences containing a zero probability.Second, learning curves are presented, that indicate the performance when using training corpora of different sizes, starting with as few as 1,000 tokens and ranging to the size of the entire corp

In [119]:
text = df["cleaned_text"][0]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][0]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['use context-independent approach for did for the contextual wts turned out good choice set all the standard deviation the unconditioned maximum likelihood probabilities the tags the training corpus, set for all using tagset tags and the average this usually yields values the range use different estimates for uppercase and lowercase words, maintain two different suffix tries depending the capitalization the word.', 'causes the probability complete sequence set zero its use necessary for new text sequence, thus makes impossible rank different sequences containing zero probability.', 'second, learning curves are presented, that indicate the performance when using training corpora different starting with few tokens and ranging the size the entire corpus important characteristic statistical taggers that they not only assign tags words but also probabilities order rank different assignments.', 'nevertheless, recent independent comparison taggers has shown that another approach even works b

In [120]:
allsum2

[['use context-independent approach for did for the contextual wts turned out good choice set all the standard deviation the unconditioned maximum likelihood probabilities the tags the training corpus, set for all using tagset tags and the average this usually yields values the range use different estimates for uppercase and lowercase words, maintain two different suffix tries depending the capitalization the word.',
  'causes the probability complete sequence set zero its use necessary for new text sequence, thus makes impossible rank different sequences containing zero probability.',
  'second, learning curves are presented, that indicate the performance when using training corpora different starting with few tokens and ranging the size the entire corpus important characteristic statistical taggers that they not only assign tags words but also probabilities order rank different assignments.',
  'nevertheless, recent independent comparison taggers has shown that another approach even 

In [122]:
reference1 = df['cleaned_summary'][0]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

[['use context-independent approach for did for the contextual wts turned out good choice set all the standard deviation the unconditioned maximum likelihood probabilities the tags the training corpus, set for all using tagset tags and the average this usually yields values the range use different estimates for uppercase and lowercase words, maintain two different suffix tries depending the capitalization the word.causes the probability complete sequence set zero its use necessary for new text sequence, thus makes impossible rank different sequences containing zero probability.second, learning curves are presented, that indicate the performance when using training corpora different starting with few tokens and ranging the size the entire corpus important characteristic statistical taggers that they not only assign tags words but also probabilities order rank different assignments.nevertheless, recent independent comparison taggers has shown that another approach even works better: mark

In [130]:
text = df["cleaned_text"][1]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][1]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['emphasize the word above, because planarity understood here does not coincide with the standard graph-theoretic concept the same name, where one would allowed also use the area below the sentence disentangle the figure shows dependency graph that planar but not projective: while there are crossing the yield the node does not form interval.', 'when compare the two graded constraints each other, find that thp degree measure partitions the data into less and larger clusters than the degree, which may advantage the context using the degree constraints features data-driven approach towards parsing.', 'parametric constraints with respect thaded constraints, find that multiplanarity different from both gap degree and degree that involves notion optimization: since every dependency graph m-planar for some sufficiently large the interesting question the context multiplanarity about the minimal values for that occur real-world data.', 'however, recent results non-projective dependency parsing,

In [131]:
text = df["cleaned_text"][2]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][2]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['leacock, towel!, and voorhees found that the response patterns the three classifiers convd, suggesting that each the classifiers was extracting much data available purely topical approaches that look only word counts from training examples.', 'leacock, towell, and voorhees showed that performance the content vector topical classifier could improved with the addition local templates— specific word patterns that were recognized being indicative particular sense— extension idea initially suggested weiss although the templates proved highly reliable when they occurred, all too often, none were found.', 'summary, the results obtained with tlc support the following preliminary conclusions: improvement with training levels off after about training examples for the least frequent sense the high predictive power local context for the verb and adjective indicate that the local parameters effectively capture syntactically mediated relations, the subject and object complement verbs, the noun tha

In [132]:
text = df["cleaned_text"][3]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][3]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['the first step apply bayes’ rule this and the second make the assumption that the features the various constituents sentence are independent given the target word and each constituent’s role and discard the term which constant with respect estimate the prior over frame element assignments the probability the frame element groups, represented with the set operator {}: and finally discard the feature prior being constant over the argmax expression: this leaves with expression terms the prior for frame element groups particular target word the local probability frame element given constituent’s features which our previous system was based, and the individual priors for the frame elements chosen this formulation can used assign roles either when the frame element boundaries are known when they are not, will discuss later this section.', 'using head word, phrase type, and target word without either position grammatical function yielded only racy, indicating that although the two features 

In [133]:
text = df["cleaned_text"][4]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][4]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['for further discussion refer the reader clark and hockenmaier ccgbank corpus ccg normal-form derivations obtained translating the penn tree bank trees using algorithm described hockenmaier and steedman almost alltypes construction?with the exception gap ping and ucp arecovered the translation procedure, which pro cesses the sentences the training corpus and the sentences the test corpus thammar contains set type-changing rules similar the lexical rules described carpenter figure shows derivation taken from ccgbank.', 'for all experiments reported here and section the frequency threshold was set likecollins assume that the test data pos tagged, and can therefore replace unknown words inthe test data with their pos-tag, which more propriate for formalism like ccg with large set lexical categories than onneric token for all unknown words.', 'this model was originally described hockenmaier where was applied preliminary version ccgbank, and its definition repeated here the top row table g

In [134]:
text = df["cleaned_text"][5]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][5]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['reproduced here for reference: given three nouns and only more recently has been suggested that corpus statistics might provide the oracle, and this idea the basis the three algorithms which use the adjacency model.', 'one problem with the training methods given section the restriction training data nouns many nouns, especially common ones, have verbal adjectival usages that preclude them from being yet when they occur nouns, they still provide useful training information that the current system ignores.', 'this result rdance with the informal reasoning given section the model also has the further commendation that predicts correctly the observed proportion left-branching compounds found two independently extracted test sets.', 'this result rdance with the informal reasoning given section the model also has the further commendation that predicts correctly the observed proportion left-branching compounds found two independently extracted test sets.']
[['reproduced here for reference: 

In [135]:
text = df["cleaned_text"][6]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][6]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['the one hand, computational linguistics volume number used annotate what kay has called permanent predictable ambiguities, allowing annotator indicate that structure isglobally ambiguous even given the surrounding context example this use pseudo-attachment shown figure where the participial phrase blown ashore years ago modifies either warriors boatload, but there way settling the question both attachments mean exactly the same thing.', 'would like emphasize that the lexical and syntactic recoverability inherent the pos-tagged version the penn treebank corpus allows end users employ much richer tagset han the small one described section the need arises.', 'about million words pos-tagged material and small sam- piing skeletally parsed text are available part the first association for com- putational linguistics/data collection initiative cd-rom, and somewhat larger subset materials available cartridge tape directly from the penn treebank project.', 'reflects not only true mistakes par

In [138]:
text = df["cleaned_text"][7]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][7]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['topic signatures can not only recognize related terms but grout related terms togetlmr under one target concept ibi iresent section pic: signatures can also shed some light the creation topic signatures.', 'the next section, describe the automated text smmnarization system summarist that used the experiments provide the context discussion.', 'experimental results order assess the utility topic signatures text sununarization, follow the procedure de- scribed the end section create topic signature for each selected trec topic.', 'the mnnber documents with answer keys are listed the row labeled: relevant does used training ensure utilize all the available data and conduct sound evaluation, perform three-fold eat three times with non- overlapl ing test set.']
[['topic signatures can not only recognize related terms but grout related terms togetlmr under one target concept ibi iresent section pic: signatures can also shed some light the creation topic signatures.the next section, describe

In [144]:
text = df["cleaned_text"][8]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
print(summary2)
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][8]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['also show that language pairs for which very little parallel data available are likely benefit the most from our method running our extraction system large comparable corpus bootstrapping manner, can obtain performance improvements more than over baseline system trained only existing parallel data.', 'there are relatively few language pairs for which parallel corpora reasonable sizes are available and even for those pairs, the corpora come mostly from one domain, that political discourse this especially problematic for the field statistical machine translation because translation systems trained data from particular domain will perform poorly when translating texts from different domain one way alleviate this lack parallel data exploit much more available and diverse resource: comparable non-parallel corpora.', 'two sentences may share many content words and yet express different meanings however, our task getting useful training data does not require perfect solution have seen, even

In [145]:
text = df["cleaned_text"][9]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][9]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

['related study, they make claim that for given discourse, ambiguous words are used single sense with high probability leacock, chodorow, and miller chall this claim their work combining topical and local context, which shows that both topical and local context are required achieve consistent results across polysemous words text yarowsky study indicates that while information within large window can used disambiguate nouns, for verbs and adjectives the size the usable window drops off dramatically with distance from the target word.', 'recent work, syntactic information most often simply part speech, used invariably conjunction with other kinds information evidence suggests that different kinds disambiguation procedures are needed depending the syntactic category and other characteristics the target word —an idea reminiscent the word expert approach.', 'step the assignment words senses, mplished reliance two major sources information: all disambiguation work involves matching the conte

In [146]:
text = df["cleaned_text"][15]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][15]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

[['interesting connection has been suggested direct parsing for id/lp grammars which word-order variations would mmodated the parser, and related ideas for generation free word-order languages the tag framework our work differs from the id/lp work several important respects.the earlier system, translation single sentences required the order hours contrast the new algorithm generally takes less than one minute—usually substantially less—with special optimization the code.the earlier system, translation single sentences required the order hours contrast the new algorithm generally takes less than one minute—usually substantially less—with special optimization the code.first approach improving the translation search limit the allowed word alignment patterns those permitted btg.']]
____________________
polynomial-time algorithm statistical machine translation introduce polynomial-time algorithm statistical machine translation. algorithm used place expensive, slow best-first search strategi

In [147]:
text = df["cleaned_text"][20]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][20]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

[['interestingly, nps are also used anchor points learn monolingual paraphrases the phrasal structure categories can extracted from automatic parsers using methods illustration given below the placement the dependency arcs reflects the relative word order between parent node and all its immediate children.all these approaches, though different formalism, model the two languages using tree-based transduction rules synchronous grammar, possibly probabilistic, and using multi-lemma elementary structures atomic units.analogous the transition probability bringing equations together, the best translation would maximize: observing the similarity between our model and hmm, our dynamic programming decoding algorithm spirit similar the viterbi algorithm except that instead being sequential the decoding done trees top down fashion.while statistical modeling children reordering one possible remedy for this problem, believe simple linguistic treatment another, the output the sdig system ish depende

In [148]:
text = df["cleaned_text"][50]
sentences = []
sentences.append(sent_tokenize(text))
sentences = [y for x in sentences for y in x]

clean_sentences = pd.Series(sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]


# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # remove stopwords from the sentences
    #clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)
        
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

    #Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary2 = []
for i in range(4):
    summary2.append(ranked_sentences[i][1])
        #summary2 += ranked_sentences[i][1]
allsum2 = []
allsum2.append(summary2)
reference1 = df['cleaned_summary'][50]
hypotheses1 = allsum2
hypotheses1 = str(hypotheses1)
hypotheses1 = re.sub("', '", '', hypotheses1)
reference1 = str(reference1)

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores1 = scorer.score(hypotheses1,
                      reference1)
print(hypotheses1)
print("____________________")
print(reference1)
print(scores1)

[['then calculated the racy counting the number words whose top one candidate identical itself, obtaining precision allowing n-top candidates, the racy improves shown thaphs for words output figure find the correct translation among the top candidates, obtain precision around n-top candidates are useful translator aids.early work uses pair non-parallel texts for the task lexical disambiguation between several senses word this basic idea extends choosing translation among multiple candidates given collocation information.evaluation shows precision about showed that humans are able translate more than twice many japanese technical terms into ish when our system output used, compared translating random set japanese terms without aid.addition the evaluation results, have also discovered that the content words the same segment with word term all contribute the occurrence this word.']]
____________________
finding terminology translations non-parallel corpora present statistical word feature